<a href="https://colab.research.google.com/github/Rutgers-Artificial-Intelligence-Club/Workshops/blob/main/Mad_Libs_Game_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Let's play an AI-Powered mad libs Game!

In [ ]:
ns

In [ ]:
# Get words from players (user inputs)
user_words = {
    "adjective1": input("Enter an adjective to describe an AI (e.g., intelligent, malfunctioning, rogue): "),
    "name": input("Enter a futuristic or sci-fi-sounding name for the AI (e.g., Nova, X-23, Syntho): "),
    "verb1": input("Enter a powerful action verb related to AI tasks (e.g., control, analyze, decode): "),
    "noun1": input("Enter a technological object or concept (e.g., mainframe, data core, neural network): "),
    "adjective2": input("Enter another adjective describing an AI tool or ability (e.g., advanced, glitchy, quantum-powered): "),
    "noun2": input("Enter a tool, weapon, or device the AI might use (e.g., laser, algorithm, drone): "),
    "noun3": input("Enter a sci-fi or dystopian threat (e.g., rogue robot, time anomaly, alien virus): "),
    "verb2": input("Enter a chaotic or unexpected action (e.g., self-destructing, hacking, mutating): "),
    "adverb1": input("Enter an adverb describing quick thinking or problem-solving (e.g., strategically, desperately, fearlessly): "),
    "verb3": input("Enter a final dramatic action (e.g., saved, reprogrammed, destroyed): "),
}

Enter an adjective to describe an AI (e.g., intelligent, malfunctioning, rogue): snarky
Enter a futuristic or sci-fi-sounding name for the AI (e.g., Nova, X-23, Syntho): HAL9000
Enter a powerful action verb related to AI tasks (e.g., control, analyze, decode): generate
Enter a technological object or concept (e.g., mainframe, data core, neural network): transformer
Enter another adjective describing an AI tool or ability (e.g., advanced, glitchy, quantum-powered): goofy
Enter a tool, weapon, or device the AI might use (e.g., laser, algorithm, drone): Samsung refrigerator
Enter a sci-fi or dystopian threat (e.g., rogue robot, time anomaly, alien virus): terminator
Enter a chaotic or unexpected action (e.g., self-destructing, hacking, mutating): fart bomb
Enter an adverb describing quick thinking or problem-solving (e.g., strategically, desperately, fearlessly): slowly
Enter a final dramatic action (e.g., saved, reprogrammed, destroyed): apocalypse


In [ ]:
# Install necessary libraries
!pip install torch transformers accelerate sentencepiece gtts

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
#!pip install torch transformers accelerate sentencepiece gtts

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

# Select an open-source model that doesn't require authentication
model_name = "microsoft/phi-2"  # Fully public, no login required!!!! :D

# Check if CUDA (GPU) is available
device = "cuda" if torch.cuda.is_available() else "cpu"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load model and move to CUDA if available
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,  # Use FP16 for GPU efficiency
    device_map="auto"  # Automatically assigns to GPU if available
)

# Create text generation pipeline WITHOUT device argument (accelerate handles it)
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Generate a dynamic AI prompt based on user input
prompt = f"""
Write an exciting sci-fi short story using these elements:
- A {user_words['adjective1']} AI named {user_words['name']}
- Mission: To {user_words['verb1']} the {user_words['noun1']}
- Armed with a {user_words['adjective2']} {user_words['noun2']}
- A sudden crisis occurs when a rogue {user_words['noun3']} starts {user_words['verb2']} uncontrollably
- Thinking {user_words['adverb1']}, {user_words['name']} takes decisive action
- In the end, the AI {user_words['verb3']} the world in an unexpected way

Make it thrilling, full of suspense, and with a dramatic ending!
"""

def generate_story(prompt):
    """ Generates an AI-powered story based on user words """
    result = generator(prompt,
                       max_length=400,  # Set a reasonable length for a short story
                       do_sample=True,
                       temperature=0.85,  # Higher for creativity
                       top_p=0.9, # top-p picks based on probability, allowing variation (e.g., sometimes 20 words, sometimes 100, depending on context).
                       top_k=50) #  top-k limits choices to a fixed number (e.g., always picking from the top 50 words).
    return result[0]['generated_text']

# AI-Generated Story
mad_libs_story = generate_story(prompt)

print("\n🦾 AI-Generated Sci-Fi Story:\n")
print(mad_libs_story)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-2-d1f5739f6d6c>", line 49, in <cell line: 0>
    mad_libs_story = generate_story(prompt)
                     ^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-2-d1f5739f6d6c>", line 40, in generate_story
    result = generator(prompt,
             ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_generation.py", line 285, in __call__
    return super().__call__(text_inputs, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/transformers/pipelines/base.py", line 1362, in __call__
    return self.run_single(inputs, preprocess_params, forward_params, postprocess_params)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/

In [ ]:
# If the text is too small or hard to read, we can have the story read aloud
from gtts import gTTS
import IPython.display as ipd

# Define the AI-generated story
story = mad_libs_story

# Convert text to speech
tts = gTTS(text=story, lang="en")
tts.save("mad_libs_story.mp3")  # Save the file

# Play audio in Colab
ipd.Audio("mad_libs_story.mp3")



Now you try! I'll share this notebook, and you can edit part of the story (story, words, AI prompt).

Extra, extra: When you're happy with your work, rename the file to your netID, and sent it to us along with your full name. We'll feature some of your work on our Social Media.